<a href="https://colab.research.google.com/github/jacksonkarel/selfmodifai/blob/main/notebooks/selfmodifai_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install huggingface_hub > /dev/null

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
import os
import json
keys = json.load(open('keys.json'))
os.environ["HUGGINGFACEHUB_API_TOKEN"] = keys['hugging_face']
import pandas as pd
import numpy as np

In [ ]:
from typing import Any, List, Mapping, Optional

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM

In [ ]:
class CustomLLM(LLM):
            
    @property
    def _llm_type(self) -> str:
        return "custom"
    
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        
        
        return prompt[:self.n]
    
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"n": self.n}

In [ ]:
llm = CustomLLM(n=10)

In [ ]:
llm("This is a foobar thing")

In [ ]:
!git clone https://github.com/tloen/alpaca-lora.git

In [ ]:
import os
os.chdir("alpaca-lora")

In [ ]:
from langchain import HuggingFaceHub

repo_id = "tloen/alpaca-lora-7b" # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options

llm = HuggingFaceHub(repo_id=repo_id, model_kwargs={"temperature":0, "max_length":64})

In [ ]:
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from langchain.utilities import PythonREPL
from langchain.agents import initialize_agent
from langchain.agents import load_tools

In [ ]:
python_repl = PythonREPL()

tools = [
    Tool(
        name="Python REPL",
        func=python_repl.run,
        description="A Python shell. Use this to execute python commands. Input should be a valid python command. Always print the output of the python command."
    ),
]

memory = ConversationBufferMemory(memory_key="chat_history")

llm=OpenAI(temperature=0)
agent_chain = initialize_agent(tools, llm, agent="conversational-react-description", verbose=True, memory=memory)

In [ ]:
tools = load_tools(["python_repl"], llm=llm)

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

input = "You are the large language model Alpaca-LoRA. Your code is in the current directory. Modify your own code so that you are a more powerful language model."

In [ ]:
agent.run(input)